In [9]:
import os
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

from catboost import CatBoostClassifier, Pool

import shap
import optuna

Bootstrapping method to find the confidence interval of the AUROC score

In [10]:
def bootstrap_auc_ci(y_true, y_scores, n_bootstraps=2000, ci=0.95):
    rng = np.random.default_rng(42)
    aucs = []

    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    for _ in range(n_bootstraps):
        idx = rng.integers(0, len(y_true), len(y_true))
        if len(np.unique(y_true[idx])) < 2:
            continue
        aucs.append(roc_auc_score(y_true[idx], y_scores[idx]))

    lower = np.percentile(aucs, (1 - ci) / 2 * 100)
    upper = np.percentile(aucs, (1 + ci) / 2 * 100)
    return np.mean(aucs), lower, upper

## Loading the dataset, pre-processing, and analysing the data

In [11]:
cohort_data = pd.read_csv('../cohort_data_new.csv')
cohort_data

,icustay_id,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,...,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,gender,icu_los_hours,target
0,200003,13.375000,3.583195,9.0,21.0,25.250000,3.105295,18.0,28.0,7.771429,...,10.0,21.0,26.471429,13.176711,13.2,43.9,48,M,141,0
1,200007,15.500000,2.121320,14.0,17.0,23.000000,1.414214,22.0,24.0,8.900000,...,8.0,10.0,10.300000,1.272792,9.4,11.2,44,M,30,0
2,200009,9.500000,2.121320,8.0,11.0,23.333333,2.081666,21.0,25.0,8.000000,...,15.0,21.0,12.471429,1.471637,10.5,14.3,47,F,51,0
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.900000,NaN,4.9,4.9,33,F,10,0
4,200014,10.000000,1.732051,9.0,12.0,24.000000,1.000000,23.0,25.0,7.733333,...,21.0,24.0,13.233333,2.203028,10.7,14.7,85,M,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30484,299992,15.375000,2.856153,11.0,25.0,23.125000,2.609556,15.0,26.0,8.307143,...,8.0,23.0,14.134783,3.781727,8.1,22.1,41,M,499,0
30485,299993,9.400000,1.341641,8.0,11.0,29.600000,2.073644,26.0,31.0,8.000000,...,12.0,15.0,12.600000,0.605530,12.0,13.3,26,M,67,0
30486,299994,16.157895,2.477973,13.0,24.0,21.631579,3.451417,17.0,31.0,8.100000,...,28.0,63.0,10.076190,2.642329,5.3,14.5,74,F,152,1
30487,299998,11.500000,1.732051,10.0,14.0,23.500000,1.290994,22.0,25.0,8.800000,...,20.0,22.0,9.900000,1.210372,7.9,11.0,87,M,46,1


In [12]:
print(f"Dataset shape: {cohort_data.shape}")
print(f"Readmission rate: {cohort_data['target'].mean() * 100:.2f}%")

Dataset shape: (30489, 93)
Readmission rate: 10.74%


In [13]:
lab_cols = [
    'anion_gap_mean', 'anion_gap_min', 'anion_gap_max', 'anion_gap_sd',
    'bicarbonate_mean', 'bicarbonate_min', 'bicarbonate_max', 'bicarbonate_sd',
    'calcium_total_mean', 'calcium_total_min', 'calcium_total_max', 'calcium_total_sd',
    'chloride_mean', 'chloride_min', 'chloride_max', 'chloride_sd',
    'creatinine_mean', 'creatinine_min', 'creatinine_max', 'creatinine_sd',
    'glucose_mean', 'glucose_min', 'glucose_max', 'glucose_sd',
    'hematocrit_mean', 'hematocrit_min', 'hematocrit_max', 'hematocrit_sd',
    'hemoglobin_mean', 'hemoglobin_min', 'hemoglobin_max', 'hemoglobin_sd',
    'mchc_mean', 'mchc_min', 'mchc_max', 'mchc_sd',
    'mcv_mean', 'mcv_min', 'mcv_max', 'mcv_sd',
    'magnesium_mean', 'magnesium_min', 'magnesium_max', 'magnesium_sd',
    'pt_mean', 'pt_min', 'pt_max', 'pt_sd',
    'phosphate_mean', 'phosphate_min', 'phosphate_max', 'phosphate_sd',
    'platelet_count_mean', 'platelet_count_min', 'platelet_count_max', 'platelet_count_sd',
    'potassium_mean', 'potassium_min', 'potassium_max', 'potassium_sd',
    'rdw_mean', 'rdw_min', 'rdw_max', 'rdw_sd',
    'red_blood_cells_mean', 'red_blood_cells_min', 'red_blood_cells_max', 'red_blood_cells_sd',
    'sodium_mean', 'sodium_min', 'sodium_max', 'sodium_sd',
    'urea_nitrogen_mean', 'urea_nitrogen_min', 'urea_nitrogen_max', 'urea_nitrogen_sd',
    'white_blood_cells_mean', 'white_blood_cells_min', 'white_blood_cells_max', 'white_blood_cells_sd',
    'age', 'icu_los_hours'
]

REmove the ICUstay_id and the gender

In [14]:
drop_cols = [c for c in cohort_data.columns if 'icustay_id' in c.lower() or 'gender' in c.lower()]
df = cohort_data.drop(columns=['icustay_id', 'gender'], errors='ignore')

X = df.drop(columns=['target'])
y = df['target']

X

,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,calcium_total_sd,...,urea_nitrogen_mean,urea_nitrogen_sd,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,icu_los_hours
0,13.375000,3.583195,9.0,21.0,25.250000,3.105295,18.0,28.0,7.771429,0.292770,...,15.571429,4.577377,10.0,21.0,26.471429,13.176711,13.2,43.9,48,141
1,15.500000,2.121320,14.0,17.0,23.000000,1.414214,22.0,24.0,8.900000,NaN,...,9.000000,1.414214,8.0,10.0,10.300000,1.272792,9.4,11.2,44,30
2,9.500000,2.121320,8.0,11.0,23.333333,2.081666,21.0,25.0,8.000000,NaN,...,17.333333,3.214550,15.0,21.0,12.471429,1.471637,10.5,14.3,47,51
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.900000,NaN,4.9,4.9,33,10
4,10.000000,1.732051,9.0,12.0,24.000000,1.000000,23.0,25.0,7.733333,0.057735,...,23.000000,1.732051,21.0,24.0,13.233333,2.203028,10.7,14.7,85,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30484,15.375000,2.856153,11.0,25.0,23.125000,2.609556,15.0,26.0,8.307143,0.255597,...,16.000000,4.662524,8.0,23.0,14.134783,3.781727,8.1,22.1,41,499
30485,9.400000,1.341641,8.0,11.0,29.600000,2.073644,26.0,31.0,8.000000,0.216025,...,13.000000,1.224745,12.0,15.0,12.600000,0.605530,12.0,13.3,26,67
30486,16.157895,2.477973,13.0,24.0,21.631579,3.451417,17.0,31.0,8.100000,0.316228,...,44.578947,12.102873,28.0,63.0,10.076190,2.642329,5.3,14.5,74,152
30487,11.500000,1.732051,10.0,14.0,23.500000,1.290994,22.0,25.0,8.800000,0.416333,...,20.750000,0.957427,20.0,22.0,9.900000,1.210372,7.9,11.0,87,46


In [17]:
# --- Train/val split ---
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.3, random_state=7, stratify=y
)

X_train, X_eval, y_train, y_eval = train_test_split(
    X_train_full, y_train_full, test_size=0.3, random_state=7, stratify=y_train_full
)
# --- CatBoost model ---
model = CatBoostClassifier(
    iterations=1500,
    learning_rate=0.03,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    l2_leaf_reg=3,
    subsample=0.8,
    verbose=200
)

# --- Train ---
model.fit(X_train, y_train, eval_set=(X_eval, y_eval), use_best_model=True)

# --- Evaluate ---
y_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_proba)

print("AUROC =", auc)
_, lower, upper = bootstrap_auc_ci(y_test, y_proba)
print(f"AUC = {auc:.4f}, 95% CI = [{lower:.4f}, {upper:.4f}]")

0:	test: 0.5667334	best: 0.5667334 (0)	total: 44.7ms	remaining: 1m 6s
200:	test: 0.6964831	best: 0.6966291 (199)	total: 4.12s	remaining: 26.6s
400:	test: 0.7046705	best: 0.7048984 (397)	total: 7.78s	remaining: 21.3s
600:	test: 0.7068951	best: 0.7072461 (590)	total: 11.8s	remaining: 17.6s
800:	test: 0.7055337	best: 0.7074231 (637)	total: 16s	remaining: 14s
1000:	test: 0.7029044	best: 0.7074231 (637)	total: 20.4s	remaining: 10.2s
1200:	test: 0.7016842	best: 0.7074231 (637)	total: 24.8s	remaining: 6.17s
1400:	test: 0.6993942	best: 0.7074231 (637)	total: 28.7s	remaining: 2.03s
1499:	test: 0.6993525	best: 0.7074231 (637)	total: 31s	remaining: 0us

bestTest = 0.707423091
bestIteration = 637

Shrink model to first 638 iterations.
AUROC = 0.7267076558226748
AUC = 0.7267, 95% CI = [0.7095, 0.7430]
